In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1321 sha256=7b8b88253d2bd633863a2fda324f0e82fa967acd5ccac468ecacae101d5edbbc
  Stored in directory: C:\Users\chuha\AppData\Local\pip\Cache\wheels\76\03\bb\589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("Data/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt',
                        "koi_fpflag_ss",
                        "koi_fpflag_co",
                        "koi_fpflag_ec",
                        "koi_period",
                        "koi_time0bk",
                        "koi_impact",
                        "koi_duration",
                        "koi_depth",
                        "koi_prad",
                        "koi_teq",
                        "koi_insol",
                        "koi_model_snr",
                       "koi_steff",
                       "koi_slogg",
                        "koi_srad",
                        "ra",
                        "dec",
                        "koi_kepmag"
                       ]].values

In [4]:
selected_features.shape

(6991, 19)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [5]:
# Get the datafarme of the target and convert it to np array
y = df.koi_disposition.values
# Encode the y values so it is ordinal instead of nominal
algo = LabelEncoder()
algo.fit(y)
y_encoded =algo.transform(y) 

In [6]:
# Split the dataset into train, test and validate with the default percentages
x_train, x_test, y_train, y_test = train_test_split(selected_features,y, random_state = np.random.randint(51))
x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, random_state = np.random.randint(51)) 

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [7]:
# Scale your data with StandardScaler
SC = MinMaxScaler()
x_train_scaled = SC.fit_transform(x_train)
x_test_scaled = SC.transform(x_test)
x_valid_scaled = SC.transform(x_valid)

# Train the Model Using KNN



In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
model = KNeighborsClassifier()
model.fit(x_train_scaled,y_train)

print(f"Training Data Score: {model.score(x_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(x_test_scaled, y_test)}")
print(f"Validating Data Score: {model.score(x_valid_scaled, y_valid)}")

Training Data Score: 0.8748807934388709
Testing Data Score: 0.7879481311975591
Validating Data Score: 0.7894736842105263


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [9]:
# Create the GridSearchCV model
# Give hyper-parameters we want to tune
para = {
    "n_neighbors" : list(range(1,50)),
    "leaf_size" : list(range(1,50)),
    "weights" : ["uniform", "distance"],
    "p" : [1,2]
}

In [10]:
# Train the model with GridSearch
model2 = GridSearchCV(KNeighborsClassifier(), para, cv = 10, n_jobs = -1)

In [11]:
%load_ext autotime
GS_results = model2.fit(x_train_scaled,y_train)

In [14]:
print(GS_results.best_params_)
print(GS_results.best_score_)

{'leaf_size': 1, 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
0.819568949074957
time: 997 µs


In [15]:
print(GS_results.best_estimator_)

KNeighborsClassifier(algorithm='auto', leaf_size=1, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                     weights='distance')
time: 1.28 ms


# Re-score with the tuned hyper-parameters

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
model = KNeighborsClassifier(n_neighbors = 18, leaf_size = 1, weights = "distance", p=1, n_jobs = -1)
model.fit(x_train_scaled,y_train)

print(f"Training Data Score: {model.score(x_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(x_test_scaled, y_test)}")
print(f"Validating Data Score: {model.score(x_valid_scaled, y_valid)}")

Training Data Score: 1.0
Testing Data Score: 0.7978642257818459
Validating Data Score: 0.8009153318077803
time: 479 ms


# Metrics(F1 score, accuracy, precision, log loss on test date)

In [24]:
from sklearn import metrics
y_predict = model.predict(x_test)
Y_predict = model.predict_proba(x_test)
# ACCURACY_test
print(f'Accuracy_test: {metrics.accuracy_score(y_test, y_predict)}')

#F1 SCORE_test
print(f'F1 Score_test: {metrics.f1_score(y_test, y_predict,average="micro")}')

# Precision_test
print(f'Precision_test: {metrics.precision_score(y_test, y_predict,average="micro")}')

# LOG LOSS_test
print(f'Log Loss_test: {metrics.log_loss(y_test, Y_predict)}')

# Recall
print(f'Recall_test: {metrics.recall_score(y_test, y_predict, average="micro")}')

Accuracy_test: 0.7238749046529367
F1 Score_test: 0.7238749046529367
Precision_test: 0.7238749046529367
Log Loss_test: 1.125464970462353
Recall_test: 0.7238749046529367
time: 224 ms


# Metrics(F1 score, accuracy, precision, log loss on validate date)

In [21]:
y_predict = model.predict(x_valid)
Y_predict = model.predict_proba(x_valid)
# ACCURACY_validate
print(f'Accuracy_validate: {metrics.accuracy_score(y_valid, y_predict)}')

#F1 SCORE_validate
print(f'F1 Score_validate: {metrics.f1_score(y_valid, y_predict,average="micro")}')

# Precision_validate
print(f'Precision_validate: {metrics.precision_score(y_valid, y_predict,average="micro")}')

# LOG LOSS_validate
print(f'Log Loss_validate: {metrics.log_loss(y_valid, Y_predict)}')

Accuracy_validate: 0.7414187643020596
F1 Score_validate: 0.7414187643020596
Precision_validate: 0.7414187643020596
Log Loss_validate: 1.222242531007512
time: 219 ms


# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)